# Malbolge Advanced Tour

This notebook demonstrates the modular malbolge package: generating programs, executing them, and inspecting profiler statistics.

In [ ]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists():
            return candidate
    raise RuntimeError("Unable to locate project root")

REPO_ROOT = find_repo_root(Path.cwd())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
from malbolge import GenerationConfig, ProgramGenerator, MalbolgeInterpreter
print("Repo root:", REPO_ROOT)

## Generate a Program
We generate a Malbolge program that prints a target string and capture profiling stats.

In [ ]:
generator = ProgramGenerator()
config = GenerationConfig(random_seed=42, max_search_depth=5, opcode_choices='op*')
target = 'Hello'
result = generator.generate_for_string(target, config=config)
result.opcodes, result.machine_output, result.stats

## Execute and Inspect
Run the opcode sequence through the interpreter and inspect the execution result.

In [ ]:
interpreter = MalbolgeInterpreter()
execution = interpreter.execute(result.opcodes, capture_machine=True)
execution.output, execution.halt_reason, execution.steps, len(execution.machine.tape) if execution.machine else None

## Profiling Helper
A simple helper to profile multiple runs and summarise heuristic stats.

In [ ]:
import time
def profile_generation(target: str, runs: int = 3):
    durations = []
    for _ in range(runs):
        start = time.perf_counter()
        res = generator.generate_for_string(target, config=config)
        durations.append(time.perf_counter() - start)
    return min(durations), sum(durations)/len(durations)
profile_generation('Hi', runs=3)

# Heuristic Comparison

Compare different generator configurations to see how heuristics affect evaluations and runtime.

In [ ]:
def run_generation(config: GenerationConfig, label: str):
    res = generator.generate_for_string(target, config=config)
    return label, res.machine_output, res.stats
configs = [
    GenerationConfig(random_seed=42, max_search_depth=5, opcode_choices='op*'),
    GenerationConfig(random_seed=42, max_search_depth=7, opcode_choices='op*j'),
]
[run_generation(cfg, f'config_{idx}') for idx, cfg in enumerate(configs, 1)]

# Interpreter Debugging

Inspect a snapshot of the machine state to debug complex programs.

In [ ]:
machine = execution.machine
tape_preview = machine.tape[:10] if machine else []
{'a': machine.a if machine else None, 'c': machine.c if machine else None, 'd': machine.d if machine else None, 'tape_preview': tape_preview}